In [ ]:
#%pip install pulp
from tkinter import *
import pulp
import networkx as nx
import numpy as np
from tkinter import messagebox
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from PIL import ImageTk,Image

In [ ]:
fen=Tk()

def fct1():
    n=int(sommet.get())
    m=int(arete.get())
    if((n<=0) or (m<=0)):
        messagebox.showerror("ERREUR","ERREUR ! n et m doivent être strictement positifs !")
        fen.destroy
    liste=saisi_matrice.get().split()
    matrice = list(map(int, liste))
    taille1=np.size(matrice)
    taille2=n*m
    if(taille1!=taille2):
        messagebox.showerror("ERREUR","matrice ne correspond pas à la taille donnée")
        fen.destroy
    incidence = np.array(matrice).reshape(n, m)
    bin=0
    for i in range(n):
        for j in range(m):
            if(incidence[i][j]!=0 and incidence[i][j]!=1):
                bin=1
                break
    if(bin==1):
        messagebox.showerror("ERREUR","ERREUR ! La matrice d'incidence sommets-arêtes prend pour valeurs 0 et 1 uniquement !")
        fen.destroy
    else :
        totalun=0
        slice_list = [slice(k, l) for k in range(0, n) for l in range(k, m)]
        results = [incidence[sl,sl] for sl in slice_list]
        for i in range (len(results)):
            d=np.linalg.det(results[i])
            if((d!=0) and (d!=1) and (d!= -1)):
                totalun=1

        if (totalun==1):
            messagebox.showinfo("PAS DE SOLUTION","La matrice n'est pas TU, le problème ne se résoud pas !")
            fen.destroy
        else:
            adjacence = (np.dot(incidence, incidence.T) > 0).astype(int)
            np.fill_diagonal(adjacence, 0)
            G=nx.from_numpy_matrix(adjacence)
            prob = pulp.LpProblem("MinimumSetVertexCover", pulp.LpMinimize)
            x = pulp.LpVariable.dicts("x", G.nodes(), cat=pulp.LpBinary)
            z = pulp.LpVariable.dicts("z", G.edges(), cat=pulp.LpBinary)
            prob += pulp.lpSum(x)
            sommet_transversal=Entry(fen,background='#f6f5f5',font=("italique",18))
            sommet_transversal.place(x=535,y=603)
            for (u,v) in G.edges():
                prob += x[u] + x[v] >= 1
            prob.solve()
            color_map = []
            for v in G.nodes():
                if pulp.value(x[v]) == 1.0 :
                    sommet_transversal.insert(0,"%s\t"%v)
                    color_map.append('#FFC107')

                else:
                    color_map.append('#f6f5f5')
            opt={
                'node_size':1000,
                'width': 2,


            }
            plt.figure()
            nx.draw(G, node_color=color_map, with_labels=True, **opt)
            plt.savefig("Graph.png", format="PNG")
            x=int(prob.objective.value())
            label4=Label(fen,text="Cardinal du transversal min = ",background='#444444',fg='#EDEDED',font=("italique",18)).place(x=160,y=500)
            cardinal_transversal=Entry(fen,background='#f6f5f5',font=("italique",18))
            cardinal_transversal.place(x=535,y=502)
            cardinal_transversal.insert(0,x)
            label5=Label(fen,text="Sommets du transversal min = ",background='#444444',fg='#EDEDED',font=("italique",18)).place(x=160,y=603)
            image_graphe = Image.open("Graph.png")
            test = ImageTk.PhotoImage(image_graphe)
            label6 =Label(image=test)
            label6.image = test
            label6.place(x=1000,y=230)


fen.geometry('1500x700+50+85')
fen.configure(background='#444444')
fen.title("TRANSVERSAL MINIMUM")
titre=Label(fen,text="Transversal Minimum",background='#444444',fg='#FFC107',font=("Mv boli",60)).place(x=300,y=10)
label1=Label(fen,text="Nombre de sommets du graphe n =",background='#444444',fg='#EDEDED',font=("italique",16) ).place(x=30,y=150)
sommet=Entry(fen,background='#f6f5f5',font=("italique",12))
sommet.place(x=390,y=155)
label2=Label(fen,text="Nombre d'arêtes du graphe m =",background='#444444',fg='#EDEDED',font=("italique",16) ).place(x=30,y=230)
arete=Entry(fen,background='#f6f5f5',font=("italique",12))
arete.place(x=390,y=235)
label3=Label(fen,text="Insérer la matrice d'incidence sommets-arêtes (les éléments sont séparés par un espace) : ",background='#444444',fg='#EDEDED',font=("italique",16)).place(x=30,y=310)
saisi_matrice=Entry(fen,background='#f6f5f5',font=("italique",18))
saisi_matrice.place(x=100,y=380)
b=Button(fen,text="Résoudre",background='#FFC107',fg='#f6f5f5',font=("italique",14),command=fct1).place(x=770,y=380)

fen.mainloop()
